In [12]:
import tensorflow
import imgaug as ia
from imgaug import augmenters as iaa
import imageio
import numpy as np
import skimage
import cv2
# import PIL
from PIL import Image
from matplotlib import pyplot as plt
from os import listdir
from os.path import isfile, join

In [10]:
images_path = "./val_images/"
labels_path = "./val_labels/"
# image_out_path = images_path + "_augmented/"
# label_out_path = labels_path + "_augmented/"
image_files = [f for f in listdir(images_path) if isfile(join(images_path, f))]
label_files = [f for f in listdir(labels_path) if isfile(join(labels_path, f))]

In [11]:
if '.DS_Store' in image_files:
    image_files.remove('.DS_Store')
if '.DS_Store' in label_files:
    label_files.remove('.DS_Store')
image_files.sort()
label_files.sort()
n_items = len(label_files)
print(n_items)

20


In [11]:
# for i in range(n_items):
for i in range(1):
    print("Processing image ", i, ": ", image_files[i])
    image = cv2.imread(join(images_path, image_files[i]))
    label = cv2.imread(join(labels_path, label_files[i]))
    
    # process 
    segmap = label.astype(np.int32)
    segmap = segmap[:,:,0]
    segmap = ia.SegmentationMapOnImage(segmap, shape=image.shape, nb_classes=1+6)
    
    # transformation types
    seq = iaa.Sequential([
    iaa.Fliplr(0.5),      
    iaa.Crop(percent=(0, 0.1)),       
    iaa.Affine(rotate=[90, 180], scale=(0.75, 1.25)),], random_order=True)
    
    images_aug = []
    segmaps_aug = []
    for _ in range(10):
#         seq_det = seq.to_deterministic()
        seq_det = seq
        img = seq_det.augment_image(image)
        img[img==0] = 255 
        images_aug.append(img)
        segmaps_aug.append(seq_det.augment_segmentation_maps([segmap])[0])
        
        
        
        

        cells = []
        for image_aug, segmap_aug in zip(images_aug, segmaps_aug):
            cells.append(image)                                      # column 1
            cells.append(segmap.draw_on_image(image))                # column 2
            cells.append(image_aug)                                  # column 3
            cells.append(segmap_aug.draw_on_image(image_aug))        # column 4
            cells.append(segmap_aug.draw(size=image_aug.shape[:2]))  # column 5

        # Convert cells to grid image and save.
        grid_image = ia.draw_grid(cells, cols=5)
        imageio.imwrite("example_segmaps.jpg", grid_image)



Processing image  0 :  1.jpg


In [44]:
# for i in range(n_items):
for i in range(1):
    print("Processing image ", i, ": ", image_files[i])
    image = cv2.imread(join(images_path, image_files[i]))
    label = cv2.imread(join(labels_path, label_files[i]))
    segmap = label.astype(np.int32)
    segmap = segmap[:,:,0]
    segmap = ia.SegmentationMapOnImage(segmap, shape=image.shape, nb_classes=1+6)
    
    # transformation types
    seq = iaa.Sequential([
    iaa.Fliplr(0.5), 
    iaa.Flipud(0.5),      
    iaa.Affine(rotate=[90, 180, 270], 
               scale={"x": (0.75, 1.25), "y": (0.75, 1.25)}, 
               translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, mode='constant', cval=0)], random_order=True)
    
    images_aug = []
    segmaps_aug = []
    for j in range(10):
        print("  Transformation ", j)
        seq_det = seq.to_deterministic()
        img = seq_det.augment_image(image)
#         img[img==0] = 255 
        images_aug.append(img)
        segmaps_aug.append(seq_det.augment_segmentation_maps([segmap])[0])
        
        image_out = image_out_path + image_files[i][:-4] + '_' + str(j) + '.jpg'
        label_out = label_out_path + image_files[i][:-4] + '_' + str(j) + '.png'
        
        # save outputs
        im = Image.fromarray(img.astype(np.uint8))
        im.save(image_out)
        seg_out = seq_det.augment_segmentation_maps([segmap])[0].get_arr_int()
#         seg_out = seq_det.augment_segmentation_maps([segmap])[0].draw_on_image(img)
        sg = Image.fromarray(seg_out.astype(np.uint8))
        sg.save(label_out)


Processing image  0 :  1.jpg
  Transformation  0
  Transformation  1
  Transformation  2
  Transformation  3
  Transformation  4
  Transformation  5
  Transformation  6
  Transformation  7
  Transformation  8
  Transformation  9
